In [ ]:
# !pip install -r requirements.txt -q
from src.training_utils import *

## **MemSum + Pegasus Encoder**

In [ ]:
!python src/src/MemSum_Full/train.py -wandb_logger True -pegasus_mode True -training_corpus_file_name src/data/PubMed/train_PUBMED_labelled.jsonl -validation_corpus_file_name src/data/PubMed/val_PUBMED.jsonl -model_folder src/model/MemSum_Full/PubMed/one_head/ -log_folder src/log/MemSum_Full/PubMed/one_head/ -vocabulary_file_name src/model/glove/vocabulary_200dim.pkl -pretrained_unigram_embeddings_file_name src/model/glove/unigram_embeddings_200dim.pkl -max_seq_len 100 -max_doc_len 100 -num_of_epochs 10 -save_every 1000 -n_device 1 -batch_size_per_device 1 -max_extracted_sentences_per_document 7 -moving_average_decay 0.999 -p_stop_thres 0.6




## **Evaluation**

In [1]:
from src.summarizers import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np
import torch

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: pubmed-summarization/section
Found cached dataset pubmed-summarization (/home/peppe/.cache/huggingface/datasets/ccdv___pubmed-summarization/section/1.0.0/f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b)
100%|██████████| 3/3 [00:00<00:00, 881.90it/s]


In [20]:
rouge_cal = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)


model_eval = 'one_head'

gpu = True #torch.cuda.is_available()

if model_eval == 'glove':
    pegasus_mode = False
    model_path = "src/model/MemSum_Full/PubMed/memsum/model_batch_1169370.pt"
    embed_dim = 200
    two_heads = False

elif model_eval == 'one_head':
    pegasus_mode = True
    model_path = "src/model/MemSum_Full/PubMed/one_head/model_batch_1003000.pt"
    embed_dim = 768
    two_heads = False
elif model_eval == 'two_heads':
    pegasus_mode = True
    model_path = "src/model/MemSum_Full/PubMed/two_heads/model_batch_735000.pt"
    embed_dim = 768
    two_heads = True





pubmed_test_data = "src/data/PubMed/test_PUBMED.jsonl"


if two_heads:
    with open("src/data/PubMed/Test_ExtAbs_PUBMED.json") as f:
        pubmed_test_data = json.load(f)


memsum_custom_data = MemSum(model_path,
                            "src/model/glove/vocabulary_200dim.pkl",
                            gpu=gpu,  max_doc_len=100, pegasus_mode=pegasus_mode, embed_dim=embed_dim, two_heads = two_heads)

In [21]:
if not two_heads:
    test_corpus_custom_data = [ json.loads(line) for line in open(pubmed_test_data)]
else:
    test_corpus_custom_data = pubmed_test_data

In [22]:
def order_sentences(shuffled_list, ordered_numbers):
    paired_list = list(zip(ordered_numbers, shuffled_list))

    sorted_list = sorted(paired_list, key=lambda x: x[0])

    ordered_sentences = [pair[1] for pair in sorted_list]

    return ordered_sentences[0]


def evaluate( model, corpus, p_stop, max_extracted_sentences, rouge_cal ):
    scores = []
    for data in tqdm(corpus):
        
        # print("original text: \n", data['text'])
        # for el in data['text']:
        #     print(el + '\n')
        
        if not two_heads:
            gold_summary = data["summary"]
            extracted_summary = model.extract( [data["text"]], p_stop_thres = p_stop, return_sentence_position = True, max_extracted_sentences_per_document = max_extracted_sentences )
        else:
            gold_summary = data[1]["summary"]
            gold_abstract = data[0]["abstract"]
            # print("Gold abstract: \n", gold_abstract)

            extracted_summary = model.extract( data, p_stop_thres = p_stop, return_sentence_position = True, max_extracted_sentences_per_document = max_extracted_sentences )
        # print("Gold summary: \n", gold_summary)
        extracted_summary = order_sentences(extracted_summary[0], extracted_summary[1])
        
        print("Extracted Summary: ", extracted_summary)
        score = rouge_cal.score( "\n".join( gold_summary ), "\n".join(extracted_summary)  )
        scores.append( [score["rouge1"].fmeasure, score["rouge2"].fmeasure, score["rougeLsum"].fmeasure ] )
        break
    return np.asarray(scores).mean(axis = 0)

In [23]:
evaluate( memsum_custom_data, test_corpus_custom_data, 0.6, 7, rouge_cal)

  0%|          | 0/6658 [00:00<?, ?it/s]

Extracted Summary:  ["anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] .", 'more specifically , we found that pd patients with anxiety were more impaired on the trail making test part b which assessed attentional set - shifting , on both digit span tests which assessed working memory and attention , and to a lesser extent on the logical memory test which assessed memory and new verbal learning compared to pd patients without anxiety . taken together ,', 'results from this study showed selective verbal memory deficits in rpd patients with anxiety compared to rpd without anxiety , whereas lpd patients with anxiety had greater attentional / working memory deficits compared to lpd without anxiety .', 'this study is the first to directly compare cognition between pd patients with and without anxiety .', 'seventeen pd patients with anxiety and thirty - three pd patients wit

array([0.5136612 , 0.18681319, 0.44808743])